In [1]:
%matplotlib inline
import matplotlib
matplotlib.style.use('fivethirtyeight')
import paramselect
import os
import fnmatch
def recursive_glob(start, pattern):
    matches = []
    for root, dirnames, filenames in os.walk(start):
        for filename in fnmatch.filter(filenames, pattern):
            matches.append(os.path.join(root, filename))
    return matches

datasets = paramselect.load_datasets(sorted(recursive_glob('Al-Ni', '*.json')))

In [2]:
from paramselect import fit, _multiphase_error
from paramselect import multi_phase_fit
from paramselect import _multiphase_fitting_system
from pycalphad import Database, equilibrium
%time dbf = fit('input.json', datasets, saveall=True)

FITTING:  BCC_B2
9 endmembers (6 distinct by symmetry)
ENDMEMBER: ('AL', 'AL', 'VA')
SYMMETRIC_ENDMEMBERS:  [('AL', 'AL', 'VA')]
ENDMEMBER: ('AL', 'NI', 'VA')
['CPM_FORM']: datasets found: 1
(T*log(T),) rss: 14.988883895 AIC: 175.267766573
(T*log(T), T**2) rss: 0.287388338954 AIC: -75.8029363477
(T*log(T), T**2, 1/T) rss: 0.00998663905176 AIC: -288.816459148
(T*log(T), T**2, 1/T, T**3) rss: 0.00726733042695 AIC: -307.159440568
['SM_FORM']: datasets found: 1
(T,) rss: 0.00256968120974 AIC: -379.694299563
['HM_FORM']: datasets found: 2
(1,) rss: 173532.435506 AIC: 786.167787589
SYMMETRIC_ENDMEMBERS:  [('AL', 'NI', 'VA'), ('NI', 'AL', 'VA')]
ENDMEMBER: ('AL', 'VA', 'VA')
['CPM_FORM']: datasets found: 0
['SM_FORM']: datasets found: 1
(T,) rss: 0.0 AIC: -inf
['HM_FORM']: datasets found: 1
(1,) rss: 0.0 AIC: -inf
SYMMETRIC_ENDMEMBERS:  [('AL', 'VA', 'VA'), ('VA', 'AL', 'VA')]
ENDMEMBER: ('NI', 'NI', 'VA')
SYMMETRIC_ENDMEMBERS:  [('NI', 'NI', 'VA')]
ENDMEMBER: ('NI', 'VA', 'VA')
['CPM_FORM']:

ValueError: shape mismatch: value array of shape (74,) could not be broadcast to indexing result of shape (66,)

In [ ]:
print(dbf.to_string(fmt='tdb', groupby='phase'))

In [ ]:
datasets = paramselect.load_datasets(sorted(recursive_glob('Al-Ni', '*.json')))
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "BCC_B2", ["AL", ["AL", "NI"], "VA"], [[0, 1]], datasets)

In [ ]:
datasets = paramselect.load_datasets(sorted(recursive_glob('Al-Ni', '*.json')))
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "BCC_B2", ["NI", ["AL", "NI"], "VA"], [[0, 1]], datasets)

In [ ]:
datasets = paramselect.load_datasets(sorted(recursive_glob('Al-Ni', '*.json')))
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "BCC_B2", ["NI", ["AL", "VA"], "VA"], [[0, 1]], datasets)

In [ ]:
datasets = paramselect.load_datasets(sorted(recursive_glob('Al-Ni', '*.json')))
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "BCC_B2", ["AL", ["NI", "VA"], "VA"], [[0, 1]], datasets)

In [ ]:
datasets = paramselect.load_datasets(sorted(recursive_glob('Al-Ni', '*.json')))
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "BCC_B2", ["VA", ["AL", "NI"], "VA"], [[0, 1]], datasets)

In [ ]:
datasets = paramselect.load_datasets(sorted(recursive_glob('Al-Ni', '*.json')))
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "BCC_B2", [["AL", "NI"], ["AL", "NI"], "VA"], [[0, 1]], datasets)

In [ ]:
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "BCC_B2", ["AL", "NI", "VA"], [[0, 1]], datasets)

In [ ]:
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "FCC_L12", ["AL", "NI", "NI", "NI", "VA"], [[0,1,2,3]], datasets)

In [ ]:
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "FCC_L12", ["AL", "AL", "NI", "NI", "VA"], [[0,1,2,3]], datasets)

In [ ]:
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "FCC_L12", ["AL", "AL", "AL", "NI", "VA"], [[0,1,2,3]], datasets)

In [ ]:
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "FCC_L12", [["AL", "NI"], "NI", "NI", "NI", "VA"], [[0,1,2,3]], datasets)

In [ ]:
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "FCC_L12", [["AL", "NI"], "NI", "NI", "AL", "VA"], [[0,1,2,3]], datasets)

In [ ]:
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "FCC_L12", [["AL", "NI"], "NI", "AL", "AL", "VA"], [[0,1,2,3]], datasets)

In [ ]:
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "FCC_L12", ["AL", ["AL", "NI"], "AL", "AL", "VA"], [[0,1,2,3]], datasets)

In [ ]:
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "FCC_L12", [["AL", "NI"], ["AL", "NI"], ["AL", "NI"], ["AL", "NI"], "VA"], [[0,1,2,3]], datasets)

In [ ]:
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "AL3NI2", ["AL", ["AL", "NI"], "NI"], None, datasets)

In [ ]:
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "AL3NI2", ["AL", ["AL", "NI"], "VA"], None, datasets)

In [ ]:
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "AL3NI2", ["AL", "NI", ["NI", "VA"]], None, datasets)

In [ ]:
paramselect.plot_parameters(dbf, ["AL", "NI", "VA"], "LIQUID", [["AL", "NI"]], None, datasets)

In [ ]:
dbf.to_file('AlNi.tdb', if_exists='overwrite')

In [ ]:
from paramselect import multi_plot
from pycalphad import binplot, Database
import pycalphad.variables as v
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(9,6))
dbf = Database('2016-04-25 15:45:29.550255-ALNI-iter4.tdb')
binplot(dbf, ['AL', 'NI', 'VA'], sorted(dbf.phases.keys()),
        {v.X('NI'): (0,1,0.02), v.T: (300, 2000, 30), v.P: 101325}, ax=fig.gca(), eq_kwargs={'_approx': True})
multi_plot(None, ['AL', 'NI', 'VA'], sorted(dbf.phases.keys()), datasets, ax=fig.gca())
plt.show()

In [ ]:
from pycalphad import calculate,Database
import pycalphad.variables as v
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
result = calculate(Database('AlNi.tdb'), ['AL', 'NI', 'VA'], ['FCC_L12', 'LIQUID'], pdens=5000, T=1645, P=1e5)

In [ ]:
fig = plt.figure(figsize=(12,12))
bcc_energies_x = result.X.sel(component='NI').values[np.nonzero(result.Phase.values == 'FCC_L12')]
bcc_energies_y = result.GM.values[np.nonzero(result.Phase.values == 'FCC_L12')]
fig.gca().scatter(bcc_energies_x, bcc_energies_y, color='b')
liq_energies_x = result.X.sel(component='NI').values[np.nonzero(result.Phase.values == 'LIQUID')]
liq_energies_y = result.GM.values[np.nonzero(result.Phase.values == 'LIQUID')]
fig.gca().scatter(liq_energies_x, liq_energies_y, color='r')
fig.gca().plot([0, 1], [-239801.46331355, -102505.10323622], color='k', linestyle='--')
#fig.gca().plot([0, 1], [-1.147e5, -6.06e4], color='k', linestyle='--')
fig.gca().set_ylim((-150000, -5e4))
fig.gca().set_xlim((0.7, 1))
#fig.gca().axvline(0.733, color='k', linestyle='--')
plt.show()